In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load Iris dataset
iris = load_iris()
X = iris.data
y = (iris.target == 0) * 2 - 1  # Convert labels to +1 and -1 (binary classification)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class AdaBoost:
    def __init__(self, n_classifiers=50):
        self.n_classifiers = n_classifiers
        self.alphas = []  # Weight of each classifier
        self.classifiers = []  # List to store the trained weak classifiers

    def fit(self, X, y):
        n_samples, n_features = X.shape
        weights = np.ones(n_samples) / n_samples  # Initialize instance weights

        for _ in range(self.n_classifiers):
            # Create a weak learner (decision stump in this case)
            classifier = DecisionStump()

            # Train the weak classifier with weighted instances
            classifier.fit(X, y, weights)

            # Make predictions
            predictions = classifier.predict(X)

            # Compute weighted error
            error = np.sum(weights * (predictions != y))

            # Calculate the weight of the classifier
            alpha = 0.5 * np.log((1 - error) / max(error, 1e-10))  # Avoid division by zero
            self.alphas.append(alpha)

            # Update instance weights
            weights *= np.exp(-alpha * y * predictions)
            weights /= np.sum(weights)  # Normalize weights

            # Save the trained weak classifier
            self.classifiers.append(classifier)

    def predict(self, X):
        # Make predictions using all weak classifiers
        classifier_predictions = [alpha * classifier.predict(X) for alpha, classifier in zip(self.alphas, self.classifiers)]
        ensemble_predictions = np.sum(classifier_predictions, axis=0)
        return np.sign(ensemble_predictions)

class DecisionStump:
    def __init__(self):
        self.feature_index = None
        self.threshold = None
        self.alpha = None

    def fit(self, X, y, weights):
        n_samples, n_features = X.shape
        min_error = float('inf')

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])

            for threshold in thresholds:
                predictions = np.ones(n_samples)
                predictions[X[:, feature] < threshold] = -1

                error = np.sum(weights * (predictions != y))

                if error < min_error:
                    min_error = error
                    self.feature_index = feature
                    self.threshold = threshold

        # Calculate the weight (alpha) of the weak learner
        self.alpha = 0.5 * np.log((1 - min_error) / max(min_error, 1e-10))

    def predict(self, X):
        n_samples = X.shape[0]
        predictions = np.ones(n_samples)
        predictions[X[:, self.feature_index] < self.threshold] = -1
        return predictions

# Create and train the AdaBoost classifier
adaboost = AdaBoost(n_classifiers=50)
adaboost.fit(X_train, y_train)

# Example prediction
predictions_train = adaboost.predict(X_train)
predictions_test = adaboost.predict(X_test)

# Calculate accuracy on training and testing sets
accuracy_train = accuracy_score(y_train, predictions_train)
accuracy_test = accuracy_score(y_test, predictions_test)

print("Accuracy on training set:", accuracy_train)
print("Accuracy on testing set:", accuracy_test)


Accuracy on training set: 0.8416666666666667
Accuracy on testing set: 0.8
